In [2]:
library(data.table)
library(dplyr)
library(BigVAR)
library(forecast)
library(DescTools)

In [3]:
setwd('/home/MCB/zkarwowska/sVAR_mbimpute/')

In [ ]:
df <- read.table('rarefied_interpolated_male_feces_OTU_112_391.csv')
df_m <-  as.matrix(df)

# create a differenced series
difference <- function(dataset){
diff = list()
for (i in (2:nrow(dataset))){
 value <- dataset[i, ] - dataset[i - 1, ]
 diff <- rbind(diff, value)
}
return(diff)
}

#invert differenced value
inverse_difference <- function(history, yhat){
#value <- yhat + history[-1, ] #interval=1
value <- yhat + tail(history, 1)
return(value)
}

#split dataset into test and train
train_X <- df[1:240, ]
test_X <- df[241:280, ]

#declare history (training set)
history <- train_X
#declare prediction matrix
predictions <- matrix(nrow = 0, ncol = 200)


#rolling forecast
for (i in (2:42)){
 print(i)
# difference data
diff <- difference(train_X)
diff_m <- as.matrix(diff)
#predict 
model <- constructModel(diff_m,
                       p=1,
                      "BasicEN",
                      gran=c(150,10),
                      RVAR=FALSE,
                      h=1,
                      cv="Rolling",
                      MN=FALSE,
                      verbose=FALSE,
                      IC=TRUE)
results=cv.BigVAR(model)
yhat <- predict(results, n.ahead=1)
yhat_t <- as.data.frame(t(c(yhat)))
yhat_undiff <- inverse_difference(history, yhat_t)

predictions <- Append(predictions, yhat_undiff, rows = 'TRUE')
# observation
obs <- test_X[i, ]
history <- Append(as.matrix(history), obs, rows = 'TRUE')

}

write.csv(predictions, 'prediction.csv', row.names = FALSE)